https://docs.python.org/3/library/multiprocessing.html

In [1]:
import time
from time import sleep
from multiprocessing import Process, Pool, Queue

In [2]:
def timer(func):
    def inner(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        elapsed = time.time() - start
        print(f'Time: {elapsed:.5f} sec ({func.__name__})')
        return res
    return inner

### Задача 1

In [3]:
# Функция, которая будет применена к каждому элементу
def square(n):
    sleep(1)
    return n * n

# Входной список значений
vals = list(range(5))

In [4]:
@timer
def sync1(values: list):
    # Список для хранения результатов
    results = []

    def worker(value):
        result = square(value)
        results.append(result)
    
    for val in values:
        worker(val)
    
    print(sum(results))

sync1(vals)

30
Time: 5.00840 sec (sync1)


In [6]:

result_queue = Queue()

# Функция для выполнения в процессах
def worker(queue, value):
    result = square(value)
    queue.put(result)


@timer
def mproc1(values: list):
    # Список для хранения результатов

    processes = []
    # Создание и запуск процессов для вызова функции square
    for value in values:
        p = Process(target=worker, args=(result_queue, value))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    gathered_results = []
    while not result_queue.empty():
        gathered_results.append(result_queue.get())

    print(sum(gathered_results))

mproc1(vals)

0
Time: 0.12781 sec (mproc1)


## Pool

In [ ]:
@timer
def mproc2(values: list):

    with Pool() as pool:
        # Use map to apply worker to a list of args
        results = pool.map(square, values)

    print(sum(results))

mproc2(vals)

#### Вопрос - почему медленнее?

### Задача 2

In [6]:
import requests

urls = [
    'http://www.python.org',
    'https://docs.python.org/3/',
    'https://docs.python.org/3/whatsnew/3.7.html',
    'https://docs.python.org/3/tutorial/index.html',
    'https://docs.python.org/3/library/index.html',
    'https://docs.python.org/3/reference/index.html',
    'https://docs.python.org/3/using/index.html',
    'https://docs.python.org/3/howto/index.html',
    'https://docs.python.org/3/installing/index.html',
    'https://docs.python.org/3/distributing/index.html',
]

def info2(results):
    print(f'results: {len(results)}, total lenght: {sum([len(i) for i in results])}')

AttributeError: Can't pickle local object 'mproc30.<locals>.worker'

In [ ]:
def worker(url):
    with requests.Session() as session:
        src = session.get(url)
    return src.content


@timer
def mproc30(_urls):
    with Pool() as pool:
        # Use map to apply worker to a list of args
        results = pool.map(worker, _urls)

    return(info2(results))

mproc30(urls)

In [39]:
# Функция для выполнения в процессе
def worker(url):
    with requests.Session() as session:
        src = session.get(url)
    return src.content
        
@timer
def mproc31(_urls):
    with Pool() as pool:
        # Use map to apply worker to a list of args
        results = pool.map(worker, _urls)

    return(info2(results))

mproc31(urls)

results: 10, total lenght: 603672
Time: 0.72486 sec (mproc31)


> processes is the number of worker processes to use.  
> If processes is None then the number returned by os.process_cpu_count()

In [5]:
import multiprocessing
import os

multiprocessing.cpu_count(), os.cpu_count()#, os.process_cpu_count()

(16, 16)

# Concurrent.futures.ProcessPoolExecutor

In [8]:
from concurrent.futures import ProcessPoolExecutor

def worker(url):
    with requests.Session() as session:
        src = session.get(url)
    return src.content


@timer
def mproc41(_urls):
    with ProcessPoolExecutor() as executor:
        
        # Using map
        results = list(executor.map(worker, _urls))
        return(info2(results))

mproc41(urls)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [53]:
from concurrent.futures import ProcessPoolExecutor


@timer
def mproc42(_urls):
    with ProcessPoolExecutor() as executor:

        # Using submit and then result()
        futures = [executor.submit(worker, i) for i in _urls]
        results = [f.result() for f in futures]

        return(info2(results))

mproc42(urls)

results: 10, total lenght: 603672
Time: 0.48732 sec (mproc4)


In [1]:
from multiprocessing import Process

def f(name):
    print('hello', name)

p = Process(target=f, args=('bob',))
p.start()
p.join()

In [2]:
from multiprocessing import Process
import os

def worker(name):
    print(f'Процесс {name} (PID: {os.getpid()}) запущен')

if __name__ == '__main__':
    processes = []
    for i in range(3):
        p = Process(target=worker, args=(f'worker-{i}',))
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()
    
    print('Все процессы завершены')

Все процессы завершены
